<a href="https://colab.research.google.com/github/friendly-besties-incorporated/all-honeypot-code/blob/main/hacs200master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import data

In [65]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime # L datetime
from pathlib import Path

mitm_out.txt, apache_out.txt, cmds_out.txt is the raw data! You can find it in the group drive.

tcpdump_output.csv is a csv (the excel).

You will need to drag and drop those into your current session (see folder icon on left sidebar).

In [66]:
mitm = pd.read_csv('mitm_out.txt', lineterminator='\n', delimiter='|')
mitm.set_axis(['containerType', 'IP', 'Time', 'attackerUser', 'attackerPwd', 'numCMDS', 'noninteractive'], axis=1, inplace=True)
mitm

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
...,...,...,...,...,...,...,...
37527,pwd-t-2_running,85.105.58.118,2022-10-24 18:12:37.473,root,1,1,y
37528,pwd-t-2_running,143.198.133.234,2022-10-25 10:04:26.467,root,1,1,y
37529,pwd-t-2_running,85.51.217.156,2022-10-25 11:55:23.683,root,1,1,y
37530,pwd-t-2_running,85.105.58.118,2022-10-24 17:56:41.127,root,1,0,n


In [67]:
apache = pd.read_csv('apache_out.txt', lineterminator='\n', delimiter='|')
apache.set_axis(['containerType', 'IP', 'Time', 'Type', 'SMTH', 'userAgent', 'noninteractive'], axis=1, inplace=True)
apache

,containerType,IP,Time,Type,SMTH,userAgent,noninteractive
0,pwd-t_running,65.154.226.109,20/Oct/2022:08:41:50,GET /resources/images/dis_logo.png HTTP/1.1,http://128.8.238.49/resources/css/footer.css,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...,y
1,pwd-t_running,167.94.138.119,21/Oct/2022:03:31:45,GET /favicon.ico HTTP/1.1,-,Mozilla/5.0 (compatible; CensysInspect/1.1; +...,y
2,pwd-t_running,45.61.185.198,20/Oct/2022:12:37:53,GET /config/getuser?index=0 HTTP/1.1,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76...,y
3,pwd-t_running,181.214.218.69,20/Oct/2022:13:04:48,GET / HTTP/1.1,-,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....,y
4,pwd-t_running,165.22.227.186,20/Oct/2022:13:34:30,\x16\x03\x01,-,-,n
...,...,...,...,...,...,...,...
1874,pwd-t-2_running,223.171.91.174,26/Oct/2022:09:59:54,GET / HTTP/1.1,${jndi:ldap://:8189/a},${jndi:ldap://:8189/a},y
1875,pwd-t-2_running,223.171.91.124,27/Oct/2022:06:12:39,GET / HTTP/1.1,${jndi:ldap://:8186/a},${jndi:ldap://:8186/a},y
1876,pwd-t-2_running,85.105.58.118,25/Oct/2022:00:07:30,GET / HTTP/1.1,${jndi:ldap://:8186/a},${jndi:ldap://:8186/a},y
1877,pwd-t-2_running,144.64.124.224,26/Oct/2022:21:53:59,GET / HTTP/1.1,${jndi:ldap://:8183/a},${jndi:ldap://:8183/a},y


In [68]:
cmds = pd.read_csv('cmds_out.txt', lineterminator='\n', delimiter='|')
cmds.set_axis(['containerType', 'IP', 'Time', 'attackerUser', 'attackerPwd', 'noninteractive', 'command'], axis=1, inplace=True)
cmds

,containerType,IP,Time,attackerUser,attackerPwd,noninteractive,command
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,y,PATH=/dev/shm/:/tmp/:./:/var/tmp/:/root/:/etc...
...,...,...,...,...,...,...,...
37573,pwd-t-2_running,120.224.143.251,2022-10-25 20:35:37.502,root,1,y,uname -a
37574,pwd-t-2_running,85.105.58.118,2022-10-24 18:12:37.473,root,1,y,uname -a
37575,pwd-t-2_running,143.198.133.234,2022-10-25 10:04:26.467,root,1,y,uname -a
37576,pwd-t-2_running,85.51.217.156,2022-10-25 11:55:23.683,root,1,y,uname -a


In [69]:
# Convert IPs to strings because strings
mitm["IP"] = mitm["IP"].astype("string")
cmds["IP"] = cmds["IP"].astype("string")
apache["IP"] = apache["IP"].astype("string")

# maybe delete trailing and leading spaces?
mitm["IP"] = mitm["IP"].replace(r"^ +| +$", r"", regex=True)
cmds["IP"] = cmds["IP"].replace(r"^ +| +$", r"", regex=True)
apache["IP"] = apache["IP"].replace(r"^ +| +$", r"", regex=True)

mitm["containerType"] = mitm["containerType"].replace(r"^ +| +$", r"", regex=True)
cmds["containerType"] = cmds["containerType"].replace(r"^ +| +$", r"", regex=True)
apache["containerType"] = apache["containerType"].replace(r"^ +| +$", r"", regex=True)

In [70]:
mitm['Time'] = pd.to_datetime(mitm['Time'], errors='coerce')
apache['Time'] = apache['Time'].apply(lambda x: dt.datetime.strptime(x, ' %d/%b/%Y:%H:%M:%S '))
cmds['Time'] = pd.to_datetime(cmds['Time'], errors='coerce')

## Functions

In [71]:
def matchApache (df, apache):
  for index, row in apache.iterrows():
    idx = df[(df['containerType'] == row['containerType']) & (df['IP'] == row['IP']) & (df['Time'] >= row['Time'])].index
    df.loc[idx, 'apacheType'] = row['Type']
    df.loc[idx, 'userAgent'] = row['userAgent']
    df.loc[idx, 'apacheTime'] = row['Time']
    df.loc[idx, 'apacheIP'] = row['IP']
    df.loc[idx, 'apacheContainer'] = row['containerType']

In [72]:
def matchTCP (df, tcp):
  for index, row in tcp.iterrows():
    idx = df[(df['containerType'] == row['containerType']) & (df['IP'] == row['IP']) & (df['Time'] >= row['Time'])].index
    df.loc[idx, 'tcpContainer'] = row['containerType']
    df.loc[idx, 'tcpTime'] = row['Time']
    df.loc[idx, 'tcpIP'] = row['IP']

In [73]:
def exportCSV (df, output):
  filepath = Path('/content/' + output + '.csv')  
  df.to_csv(filepath)

# Apache first check (different container types for 1/2)

In [74]:
apacheMatch = mitm.loc[(mitm['IP'].isin(apache['IP'])) & (mitm['containerType'].isin(apache['containerType']))]
apacheMatch = apacheMatch.reset_index()
apacheMatch = apacheMatch.drop(columns='index')
apacheMatch

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
3,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
4,pwd-t_running,84.16.224.173,2022-10-23 21:09:01.608,root,1,1,y
...,...,...,...,...,...,...,...
21564,pwd-t-2_running,85.105.58.118,2022-10-24 18:12:37.473,root,1,1,y
21565,pwd-t-2_running,143.198.133.234,2022-10-25 10:04:26.467,root,1,1,y
21566,pwd-t-2_running,85.51.217.156,2022-10-25 11:55:23.683,root,1,1,y
21567,pwd-t-2_running,85.105.58.118,2022-10-24 17:56:41.127,root,1,0,n


In [75]:
matchApache(apacheMatch, apache)

## Output

In [76]:
usable1 = apacheMatch.dropna()
usable1

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive,apacheType,userAgent,apacheTime,apacheIP,apacheContainer
1469,it-t-2_running,173.18.35.41,2022-10-23 01:12:18.451,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1490,it-t-2_running,173.18.35.41,2022-10-24 03:19:26.313,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1501,it-t-2_running,173.18.35.41,2022-10-23 13:15:57.007,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1560,it-t-2_running,173.18.35.41,2022-10-23 07:28:02.403,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
1594,it-t-2_running,173.18.35.41,2022-10-23 15:33:54.631,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-22 05:02:27,173.18.35.41,it-t-2_running
...,...,...,...,...,...,...,...,...,...,...,...,...
21557,pwd-t-2_running,31.14.115.42,2022-10-25 11:35:15.920,root,1,0,n,GET / HTTP/1.1,${jndi:ldap://:8183/a},2022-10-25 01:04:18,31.14.115.42,pwd-t-2_running
21560,pwd-t-2_running,89.100.220.200,2022-10-25 09:04:21.442,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8180/a},2022-10-25 02:55:07,89.100.220.200,pwd-t-2_running
21563,pwd-t-2_running,120.224.143.251,2022-10-25 20:35:37.502,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8183/a},2022-10-25 00:45:21,120.224.143.251,pwd-t-2_running
21564,pwd-t-2_running,85.105.58.118,2022-10-24 18:12:37.473,root,1,1,y,GET / HTTP/1.1,${jndi:ldap://:8186/a},2022-10-24 18:06:52,85.105.58.118,pwd-t-2_running


In [ ]:
exportCSV(usable1, 'outApache1')

In [77]:
display(pd.unique(usable1['IP']))
display(len(pd.unique(usable1['IP'])))

<StringArray>
[   '173.18.35.41',  '223.171.91.176',  '223.171.91.163',   '78.187.13.206',
    '40.87.11.253',  '89.100.220.200',    '69.236.215.6',   '117.16.44.111',
  '182.224.177.28',  '182.224.177.33', '143.198.133.234',   '195.87.73.208',
 '102.141.225.238',  '222.117.95.174',   '85.105.58.118',  '223.171.91.149',
  '129.154.55.234',  '120.211.227.11',  '91.125.119.231',    '31.165.5.155',
  '157.245.137.18',  '223.99.166.104', '118.218.209.149',    '31.14.115.42',
  '147.182.233.56', '120.224.143.251',  '146.56.115.253',   '222.103.98.58',
  '223.171.91.120',  '222.165.136.99',   '118.41.204.72', '223.223.200.243',
  '223.171.91.169',  '162.19.210.221',  '223.171.91.148',  '222.134.240.91',
  '223.171.91.173',   '85.51.217.156',  '120.236.79.182',   '101.35.232.15',
    '1.220.98.197',   '84.16.224.173',  '223.171.91.161',   '82.64.113.245',
    '58.216.8.121',  '144.22.191.115',    '93.91.117.60',    '54.38.97.156',
  '184.67.149.186',   '103.105.12.48',   '78.189.25.224',   '8

83

In [ ]:
filteredCMDS1 = cmds[(cmds['IP'].isin(usable1['IP'])) & (cmds['containerType'].isin(usable1['containerType'])) & (cmds['Time'].isin(usable1['Time']))]
filteredCMDS1

In [ ]:
exportCSV(filteredCMDS1, 'outApacheCMDS1')

# TCP first check (different container types for 1/2)

In [78]:
tcp = pd.read_csv('tcpdump_output.csv', lineterminator='\n', delimiter=',')
tcp.set_axis(['index', 'IP', 'Time', 'containerType'], axis=1, inplace=True)
tcp = tcp.drop(columns='index')

tcp['Time'] = tcp['Time'].apply(lambda x : dt.datetime.utcfromtimestamp(x))
tcp["IP"] = tcp["IP"].astype("string")
tcp["IP"] = tcp["IP"].replace(r"^ +| +$", r"", regex=True)

tcp["containerType"] = tcp["containerType"].replace(r"^ +| +$", r"", regex=True)

tcp['containerType'] = tcp['containerType'].astype(str) + "_running"

tcp

,IP,Time,containerType
0,175.107.13.124,2022-10-20 22:49:33.245104,pwd-t_running
1,175.107.13.124,2022-10-20 22:49:33.471893,pwd-t_running
2,175.107.13.124,2022-10-20 22:49:33.756950,pwd-t_running
3,175.107.13.124,2022-10-20 22:49:33.985836,pwd-t_running
4,175.107.13.124,2022-10-20 22:49:33.989256,pwd-t_running
...,...,...,...
228136,45.139.105.208,2022-10-23 18:50:06.489048,pwd-t-2_running
228137,45.139.105.208,2022-10-23 18:50:06.491150,pwd-t-2_running
228138,45.139.105.208,2022-10-23 18:50:06.583090,pwd-t-2_running
228139,45.139.105.208,2022-10-23 18:50:06.583351,pwd-t-2_running


In [79]:
tcpMatch = mitm.loc[(mitm['IP'].isin(tcp['IP'])) & (mitm['containerType'].isin(tcp['containerType']))]
tcpMatch = tcpMatch.reset_index()
tcpMatch = tcpMatch.drop(columns='index')
tcpMatch

,containerType,IP,Time,attackerUser,attackerPwd,numCMDS,noninteractive
0,pwd-t_running,173.18.35.41,2022-10-24 06:29:53.320,git,12345qwert,1,y
1,pwd-t_running,191.242.188.103,2022-10-23 17:47:57.987,git,12345qwert,1,y
2,pwd-t_running,209.216.177.158,2022-10-24 04:23:02.961,git,12345qwert,1,y
3,pwd-t_running,31.165.5.151,2022-10-23 11:36:31.614,git,12345qwert,1,y
4,pwd-t_running,91.125.119.231,2022-10-23 21:35:27.480,git,12345qwert,1,y
...,...,...,...,...,...,...,...
29553,pwd-t-2_running,85.105.58.118,2022-10-24 18:12:37.473,root,1,1,y
29554,pwd-t-2_running,143.198.133.234,2022-10-25 10:04:26.467,root,1,1,y
29555,pwd-t-2_running,85.51.217.156,2022-10-25 11:55:23.683,root,1,1,y
29556,pwd-t-2_running,85.105.58.118,2022-10-24 17:56:41.127,root,1,0,n


In [80]:
tcpShorter = tcp.loc[(tcp['IP'].isin(mitm['IP'])) & (tcp['containerType'].isin(mitm['containerType']))]
tcpShorter

,IP,Time,containerType
692,180.93.32.58,2022-10-26 19:00:19.082565,pwd-t_running
693,180.93.32.58,2022-10-26 19:00:19.334816,pwd-t_running
694,180.93.32.58,2022-10-26 19:00:19.337175,pwd-t_running
695,180.93.32.58,2022-10-26 19:00:19.337175,pwd-t_running
699,180.93.32.58,2022-10-26 19:04:08.251953,pwd-t_running
...,...,...,...
228052,27.1.44.56,2022-10-23 09:13:39.224025,pwd-t-2_running
228053,27.1.44.56,2022-10-23 09:13:39.224025,pwd-t-2_running
228054,27.1.44.56,2022-10-23 09:13:44.260613,pwd-t-2_running
228055,27.1.44.56,2022-10-23 09:13:59.241195,pwd-t-2_running


In [ ]:
matchTCP(tcpMatch, tcpShorter)

## Output

In [ ]:
usableTCP = tcpMatch.dropna()
usableTCP

In [ ]:
exportCSV(usableTCP, 'outTCP1')

In [ ]:
filteredTCPCMDS = cmds[(cmds['IP'].isin(usableTCP['IP'])) & (cmds['containerType'].isin(usableTCP['containerType'])) & (cmds['Time'].isin(usableTCP['Time']))]
filteredTCPCMDS

In [ ]:
exportCSV(filteredTCPCMDS, 'outTCPCMDS1')

In [ ]:
filteredTCPCMDS.groupby(by='command').count()

In [ ]:
usableTCP.groupby(by='IP').count()
usableTCP.groupby(by='attackerUser').count()
usableTCP.groupby(by='attackerPwd').count()

# Apache second check (same container types for 1/2)

consider 1/2 the same container types to see if it affects results

For example, swipe-t_running and swipe-t-2_running are now considered the same type

In [ ]:
# cursed code
mitm['containerType'] = mitm['containerType'].str.replace('swipe-t_running','swipe')
mitm['containerType'] = mitm['containerType'].str.replace('swipe-t-2_running','swipe')
mitm['containerType'] = mitm['containerType'].str.replace('it-t_running','it')
mitm['containerType'] = mitm['containerType'].str.replace('it-t-2_running','it')
mitm['containerType'] = mitm['containerType'].str.replace('blank-t_running','blank')
mitm['containerType'] = mitm['containerType'].str.replace('blank-t-2_running','blank')
mitm['containerType'] = mitm['containerType'].str.replace('control-t_running','control')
mitm['containerType'] = mitm['containerType'].str.replace('pwd-t_running','pwd')
mitm['containerType'] = mitm['containerType'].str.replace('pwd-t-2_running','pwd')

apache['containerType'] = apache['containerType'].str.replace('swipe-t_running','swipe')
apache['containerType'] = apache['containerType'].str.replace('swipe-t-2_running','swipe')
apache['containerType'] = apache['containerType'].str.replace('it-t_running','it')
apache['containerType'] = apache['containerType'].str.replace('it-t-2_running','it')
apache['containerType'] = apache['containerType'].str.replace('blank-t_running','blank')
apache['containerType'] = apache['containerType'].str.replace('blank-t-2_running','blank')
apache['containerType'] = apache['containerType'].str.replace('control-t_running','control')
apache['containerType'] = apache['containerType'].str.replace('pwd-t_running','pwd')
apache['containerType'] = apache['containerType'].str.replace('pwd-t-2_running','pwd')

cmds['containerType'] = cmds['containerType'].str.replace('swipe-t_running','swipe')
cmds['containerType'] = cmds['containerType'].str.replace('swipe-t-2_running','swipe')
cmds['containerType'] = cmds['containerType'].str.replace('it-t_running','it')
cmds['containerType'] = cmds['containerType'].str.replace('it-t-2_running','it')
cmds['containerType'] = cmds['containerType'].str.replace('blank-t_running','blank')
cmds['containerType'] = cmds['containerType'].str.replace('blank-t-2_running','blank')
cmds['containerType'] = cmds['containerType'].str.replace('control-t_running','control')
cmds['containerType'] = cmds['containerType'].str.replace('pwd-t_running','pwd')
cmds['containerType'] = cmds['containerType'].str.replace('pwd-t-2_running','pwd')

tcp['containerType'] = tcp['containerType'].str.replace('swipe-t_running','swipe')
tcp['containerType'] = tcp['containerType'].str.replace('swipe-t-2_running','swipe')
tcp['containerType'] = tcp['containerType'].str.replace('it-t_running','it')
tcp['containerType'] = tcp['containerType'].str.replace('it-t-2_running','it')
tcp['containerType'] = tcp['containerType'].str.replace('blank-t_running','blank')
tcp['containerType'] = tcp['containerType'].str.replace('blank-t-2_running','blank')
tcp['containerType'] = tcp['containerType'].str.replace('control-t_running','control')
tcp['containerType'] = tcp['containerType'].str.replace('pwd-t_running','pwd')
tcp['containerType'] = tcp['containerType'].str.replace('pwd-t-2_running','pwd')

In [ ]:
apacheMatch2 = mitm.loc[(mitm['IP'].isin(apache['IP'])) & (mitm['containerType'].isin(apache['containerType']))]
apacheMatch2 = apacheMatch2.reset_index()
apacheMatch2 = apacheMatch2.drop(columns='index')
apacheMatch2

In [ ]:
matchApache(apacheMatch2, apache)

## Output

In [ ]:
usable2 = apacheMatch2.dropna()
usable2

In [ ]:
display(pd.unique(usable2['IP']))
display(len(pd.unique(usable2['IP'])))
okIPs=usable2['IP']

In [ ]:
filteredCMDS = cmds[(cmds['IP'].isin(usable2['IP'])) & (cmds['containerType'].isin(usable2['containerType'])) & (cmds['Time'].isin(usable2['Time']))]
filteredCMDS

In [ ]:
filteredCMDS.groupby(by='command').count()

In [ ]:
usable2.groupby(by='IP').count()

In [ ]:
usable2.groupby(by='attackerUser').count()

In [ ]:
usable2.groupby(by='attackerPwd').count()

# TCP second check (same container types for 1/2)

In [ ]:
tcpMatch2 = mitm.loc[(mitm['IP'].isin(tcp['IP'])) & (mitm['containerType'].isin(tcp['containerType']))]
tcpMatch2 = tcpMatch2.reset_index()
tcpMatch2 = tcpMatch2.drop(columns='index')
tcpMatch2

In [ ]:
tcpShorter2 = tcp.loc[(tcp['IP'].isin(mitm['IP'])) & (tcp['containerType'].isin(mitm['containerType']))]
tcpShorter2

In [ ]:
matchTCP(tcpMatch2, tcpShorter2)

In [ ]:
usableTCP2 = tcpMatch2.dropna()
usableTCP2

## Output

In [ ]:
exportCSV(usableTCP2, 'outTCP2')

In [ ]:
filteredTCPCMDS2 = cmds[(cmds['IP'].isin(usableTCP2['IP'])) & (cmds['containerType'].isin(usableTCP2['containerType'])) & (cmds['Time'].isin(usableTCP2['Time']))]
filteredTCPCMDS2

In [ ]:
exportCSV(filteredTCPCMDS2, 'outTCPCMDS2')

In [ ]:
filteredTCPCMDS2.groupby(by='command').count()

In [ ]:
usableTCP2.groupby(by='IP').count()

In [ ]:
usableTCP2.groupby(by='attackerUser').count()

In [ ]:
usableTCP2.groupby(by='attackerPwd').count()